# Geometric Gauges: Plane and Simple

In [1]:
%pip install -q kingdon anywidget==0.9.13

Note: you may need to restart the kernel to use updated packages.


In [12]:
from kingdon import Algebra
import numpy as np
import itertools
import timeit
import ipywidgets as ipy

alg = Algebra(2, 0, 1)
locals().update(alg.blades)

options = dict(
    lineWidth=4,
    pointRadius=2.5,
    fontSize=3,
    scale=1,
)
animated_options = dict(animate=0, **options)
clrs = [0xff9900, 0xfed290, 0x009977]

In [13]:
# Construct points and lines from coefficients
point = lambda x, y: (e0 + x*e1 + y*e2).dual()
line = lambda a, b, c: a*e1 + b*e2 + c*e0

# The formulas for projection and rejection are the same for all elements of geometry in PGA
project = lambda a, b: (a | b) / b
reject = lambda a, b: (a | b)

In [14]:
# https://enkimute.github.io/ganja.js/examples/coffeeshop.html#XOWMGemyJ

# Initiate a shape
np.random.seed(42)
coords = np.ones((3, 5))
coords[1:] = np.random.uniform(0.5, 1.5, size=(2, 5))
points1 = alg.vector(coords).dual()
points2 = points1.map(lambda v: np.roll(v, 1, axis=-1))
shape = list(zip(points1, points2))

# Reflections
origin = alg.blades.e0.dual()
p1 = alg.vector(e0=1, e1=0, e2=0).dual()
p2 = alg.vector(e0=1, e1=0, e2=1).dual()
p3 = alg.vector(e0=1, e1=1, e2=0.3).dual()
u = lambda: (p1 & p2).normalized()
L1 = alg.vector(e1=1).normalized()
L2 = alg.vector(e1=1, e2=0.5).normalized()
L3 = alg.vector(e1=1, e0=0.5).normalized()
L4 = alg.vector(e2=1).normalized()

line_point_end = alg.vector(e0=1, e1=2, e2=0).dual().normalized()
line_point_begin = alg.vector(e0=2, e1=2, e2=0).dual().normalized()
line_segment = [line_point_begin, line_point_end]

pa = alg.vector(e0=1, e1=0.5, e2=0).dual()
pb = alg.vector(e0=1, e1=-1.5, e2=0).dual()

## Introduction

- About Reflections

# About Reflections

In $1$ dimension, a single reflection .... Two point reflections on the line make a translation along the line.

In [16]:
refl_1d_slider = ipy.IntSlider(
    value=0,
    min=0,
    max=2,
    step=1,
    description='Reflections:',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='d',
)

def refl_1d_graph_func():
    line_segment_a = [pa >> point for point in line_segment]
    line_segment_b = [pb >> point for point in line_segment_a]

    if refl_1d_slider.value == 0:
        return [
            L4,
            pa, 'a',
            '<G stroke-width="0.05">',
            clrs[0],
            line_segment,
            line_segment[-1],
            '</G>',
        ]
    elif refl_1d_slider.value == 1:
        return [
            L4,
            pa, 'a',
            '<G stroke-width="0.05">',
            clrs[0],
            line_segment,
            line_segment[-1],
            '</G>',
            '<G stroke-width="0.05">',
            clrs[2],
            line_segment_a,
            line_segment_a[-1],
            '</G>',
        ]
    elif refl_1d_slider.value == 2:
        return [
            L4,
            pa, 'a',
            pb, 'b',
            '<G stroke-width="0.05">',
            clrs[0],
            line_segment,
            line_segment[-1],
            '</G>',
            '<G stroke-width="0.05">',
            clrs[2],
            line_segment_a,
            line_segment_a[-1],
            '</G>',
            '<G stroke-width="0.05">',
            clrs[0],
            line_segment_b,
            line_segment_b[-1],
            '</G>',
        ]

refl_1d_graph = alg.graph(
    refl_1d_graph_func, **animated_options
)

ipy.VBox([refl_1d_slider, refl_1d_graph])

As we go to higher dimensions, we would still like two point reflections to create a translation along the line.

In $d$ dimensions, a single reflection inverts the entire space except for the $d-1$ dimensional subspace in which we reflect.

In [6]:
def refl_graph_func():
    t = timeit.default_timer() / 50
    # Create the reflected shape and the lines between them
    R = np.cos(t) + origin*np.sin(t)
    L1p = R >> L1
    _rpoints1 = L1p >> points1
    _rpoints2 = L1p >> points2
    rshape = zip(_rpoints1, _rpoints2)
    rlines = zip(_rpoints1, points1)
    
    return [
        L1p,
        clrs[0],
        *shape, 
        clrs[2],
        *rshape,
        '<G stroke-width="0.002">',clrs[0],
        *rlines,
        '</G>',
    ]

refl_graph = alg.graph(
    refl_graph_func, **animated_options
)
refl_graph

GraphWidget(cayley=[['1', 'e0', 'e1', 'e2', 'e01', 'e02', 'e12', 'e012'], ['e0', '0', 'e01', 'e02', '0', '0', …

## Bireflections

Two reflections can form a translation, a rotation, or a boost. These are all examples of **bireflections**.

In [7]:
def bi_refl_graph_func():
    # Reflect the points once and then once more.
    rpoints1 = u >> points1
    rrpoints1 = L1 >> rpoints1
    rpoints2 = u >> points2
    rrpoints2 = L1 >> rpoints2
    
    # Create the reflected shape and the lines between them
    rshape = zip(rpoints1, rpoints2)
    rrshape = zip(rrpoints1, rrpoints2)
    rlines = zip(rpoints1, points1)
    rrlines = zip(rpoints1, rrpoints1)
    
    return [
        p1, p2,
        u,
        L1,
        clrs[0],
        *shape, 
        '<G stroke-dasharray="0.02 0.02">',clrs[1],
        *rshape,
        '</G>',
        '<G stroke-width="0.002">',clrs[0],
        *rlines,
        *rrlines,
        '</G>',
        clrs[2], 
        *rrshape,
    ]

def point_graph_func():
    # Create the reflected shape and the lines between them
    point_reflected_points1 = p1 >> points1
    point_reflected_points2 = p1 >> points2
    rshape = zip(point_reflected_points1, point_reflected_points2)
    rlines = zip(point_reflected_points1, points1)
    
    return [
        p1,
        clrs[0],
        *shape, 
        clrs[2],
        *rshape,
        '<G stroke-width="0.002">',clrs[0],
        *rlines,
        '</G>',
    ]

In [8]:
bi_refl_graph = alg.graph(
    bi_refl_graph_func,
    **options
)

point_graph = alg.graph(
    point_graph_func,
    **options
)

# bi_refl_graph
grid = ipy.GridspecLayout(1, 2)
grid[0, 0] = bi_refl_graph
grid[0, 1] = point_graph
grid

GridspecLayout(children=(GraphWidget(cayley=[['1', 'e0', 'e1', 'e2', 'e01', 'e02', 'e12', 'e012'], ['e0', '0',…

- Identity is the special case of two identical reflections $\implies$ Reflections are *involutary*.
  > Doing the same reflection twice is the same as doing nothing

- Two orthogonal reflections are another special case, which is identical to a point reflection.
  > A point(-reflection) **is** two orthogonal line(-reflection)s.

<p style="text-align: center;">All bireflections are <b>boring</b> plus <b>drastic</b>.</p>

## Bireflections have Geometric Gauges

Given the input and output shape, you could reconstruct the bireflection between them. However, you could never know which specific reflections were used to make that bireflection! 
**<p style="text-align: center;">Bireflections have a gauge degree of freedom!</p>**

In [9]:
def graph_func(L1, L2):
    intersection = L1 ^ L2
    if not intersection**2:
        R = lambda t: 1 + intersection*np.sin(5*t)
    else:
        R = lambda t: np.cos(t) + intersection*np.sin(t)
        
    def _graph_func():
        t = timeit.default_timer() / 25
        L1p = R(t) >> L1
        L2p = R(t) >> L2
        # Reflect the points once and then once more.
        rpoints1 = L1p >> points1
        rrpoints1 = L2p >> rpoints1
        rpoints2 = L1p >> points2
        rrpoints2 = L2p >> rpoints2
        # Create the reflected shape and the lines between them
        rshape = zip(rpoints1, rpoints2)
        rrshape = zip(rrpoints1, rrpoints2)
        rlines = zip(rpoints1, points1)
        rrlines = zip(rpoints1, rrpoints1)
        
        return [
            L1p,
            L2p,
            0xff9900,
            *shape, 
            '<G stroke-dasharray="0.02 0.02">',0xfed290,
            *rshape,
            '</G>',
            '<G stroke-width="0.002">',0xff9900,
            *rlines,
            *rrlines,
            '</G>',
            0x009977, 
            *rrshape,
        ]
    return _graph_func
    
rot_graph = alg.graph(
    graph_func(L1, L2),
    **animated_options
)

trans_graph = alg.graph(
    graph_func(L1, L3),
    **animated_options
)

grid = ipy.GridspecLayout(1, 2)
grid[0, 0] = rot_graph
grid[0, 1] = trans_graph
grid

GridspecLayout(children=(GraphWidget(cayley=[['1', 'e0', 'e1', 'e2', 'e01', 'e02', 'e12', 'e012'], ['e0', '0',…

## Cartan-Dieudonné theorem

**Geometric Gauges** are the geometric mechanism behind the famous *Cartan-Dieudonné theorem*:
> Every orthogonal transformation in an $n$-dimensional embedding space is composed from at most $n$ reflections. 

<iframe src="https://enkimute.github.io/ganja.js/examples/coffeeshop.html#Mqs0ezNVP&fullscreen" width="100%" height="100%" frameBorder="0" scrolling="no"></iframe>

<p style="text-align: center;">In fact: <b>Geometric Gauges explain all Products!</b></p>

## Conjugation (a.k.a. Sandwich Product)

Gauges explain the conjugation formula $aba^{-1}$ for $a,b \in \mathcal{G}$. 
Let's reflect the line $b$ in the line $a$: $aba$.
- Use the gauge degree of freedom in $ab = a'b'$ until $b' = a$, at which point $ a b a = (a' b') a  = a' ( b' a) = a'$

In [10]:
sw_slider = ipy.FloatSlider(
    value=0.0,
    min=-1.0,
    max=1.0,
    step=0.05,
    description='',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

def sw_graph_func():
    if sw_slider.value == 0.0:
        return [
            clrs[0],
            L1, 'a',
            clrs[2],
            L2, "b'",
            '<G stroke-dasharray="0.1 0.1">', clrs[1],
            L1, "a'",
            '</G>',
        ]
    elif sw_slider.value < 1.0:
        t = np.arccos((L1 | L2).e) * sw_slider.value
        intersection = L1 ^ L2
        R = np.cos(t) + intersection*np.sin(t)
        L2p = R >> L2
        L1p = R >> L1
        
        return [
            clrs[0],
            L1, 'a',
            clrs[2],
            L2p, "b'",
            clrs[1],
            L1p, "a'",
        ]
    else:
        R = (L1 * L2).sqrt()
        L2p = R >> L2
        L1p = R >> L1
        
        return [
            '<G fill-opacity="0.3" stroke-opacity="0.1">', 
            clrs[0],
            L1, 'a',
            '</G>',
            '<G stroke-opacity="0.3" fill-opacity="0.3" stroke-dasharray="0.1 0.1">', 
            clrs[2],
            L2p, "b'",
            '</G>',
            clrs[0],
            L1p, "a'",
        ]
    
sw_graph = alg.graph(
    sw_graph_func,
    **animated_options,
)


ipy.VBox([sw_slider, sw_graph])

## Product between two points

The product between two points $a$ and $b$: $R = ab$.

In [11]:
p1_slider = ipy.FloatSlider(
    value=0.0,
    min=-1.0,
    max=1.0,
    step=0.05,
    description='',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)
p2_slider = ipy.FloatSlider(
    value=0.0,
    min=-1.0,
    max=1.0,
    step=0.05,
    description='',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

p1l1 = alg.vector(e0=1.0, e1=1.0, e2=0.0).normalized()
p1l2 = alg.vector(e0=1.0, e1=0.0, e2=1.0).normalized()
p1 = p1l1 ^ p1l2
p2l1 = alg.vector(e0=-1.0, e1=1.0, e2=0.0).normalized()
p2l2 = alg.vector(e0=-1.0, e1=0.0, e2=1.0).normalized()
p2 = p2l1 ^ p2l2

def points_gp_graph_func():
    R1 = np.cos(0.25*np.pi*p1_slider.value) + p1*np.sin(0.25*np.pi*p1_slider.value)
    R2 = np.cos(0.25*np.pi*p2_slider.value) + p2*np.sin(0.25*np.pi*p2_slider.value)
    _p1l1 = R1 >> p1l1
    _p1l2 = R1 >> p1l2
    _p2l1 = R2 >> p2l1
    _p2l2 = R2 >> p2l2
    
    _p1 = _p1l1 ^ _p1l2
    _p2 = _p2l1 ^ _p2l2
    if abs((_p1l1 ^ _p2).dual().e) < 1e-3 and abs((_p2l1 ^ _p1).dual().e) < 1e-3:
        return [
            clrs[0],
            _p1l2, _p1, 'a',
            '<G stroke-opacity="0.1">',
            _p1l1,
            '</G>',
            clrs[2],
            _p2l2, _p2, "b",
            '<G stroke-dasharray="0.1 0.1" stroke-opacity="0.1">',
            _p2l1,
            '</G>',
        ]
    return [
        clrs[0],
        _p1l1, _p1l2, _p1, 'a',
        clrs[2],
        _p2l1, _p2l2, _p2, "b",
    ]
    
points_gp_graph = alg.graph(
    points_gp_graph_func,
    **options, animate=1,
)


ipy.VBox([p1_slider, p2_slider, points_gp_graph])

The product of two points is a translation along the line $a \vee b$! In fact, it is **twice** the translation from $a$ to $b$.

## Algebra of Geometry

<iframe src="https://enkimute.github.io/ganja.js/examples/coffeeshop.html#8lN3hOoNj&fullscreen" width="100%" height="100%" frameBorder="0" alt="iframe"></iframe>

# Invariant Decomposition

<iframe src="https://enkimute.github.io/ganja.js/examples/coffeeshop.html#ffeKS-07k&fullscreen" width="100%" height="100%" frameBorder="0" alt="iframe"></iframe>

fragement1

fragement2

fragement3

## First Principle

## Invariant Decomposition

$$ b_i = \frac{\lambda_i B + \tfrac{1}{3!} B \wedge B \wedge B}{\lambda_i + \tfrac{1}{2} B \wedge B} $$

## Conclusion

Live demo? This entire presentation has been a live demo!
End by qouting Charles like in the GSG paper?

<iframe src="https://enkimute.github.io/ganja.js/examples/coffeeshop.html#XOWMGemyJ&fullscreen" width="100%" height="100%" frameBorder="0" alt="iframe"></iframe>